In [19]:
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
!unzip -qq "/content/gdrive/MyDrive/유전체 정보 품종 분류 AI 경진대회.zip"

In [20]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [21]:
def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class','father','mother','gender'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id','father','mother','gender'])
        return df_x

In [22]:
train_x, train_y = get_x_y(train)
test_x = get_x_y(test)

In [23]:
train_x

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,2,G G,A G,A A,G A,C A,A A,A A,G G,A A,G G,A G,A A,A A,A A,A A
1,2,A G,A G,C A,A A,A A,A G,A A,G A,A A,A G,A A,G A,G G,A A,A A
2,2,G G,G G,A A,G A,C C,G G,A A,G A,G A,A G,A A,A A,A A,A A,A A
3,1,A A,G G,A A,G A,A A,G G,G G,A A,G G,A G,G G,G G,G G,A A,G G
4,2,G G,G G,C C,A A,C C,A A,A A,A A,A A,G G,A A,A A,A G,A A,G A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,2,A G,A G,A A,G A,C C,A G,A A,G A,A A,G G,A G,G A,A A,A A,A A
258,2,G G,A A,C A,A A,A A,A G,G A,G A,A A,A G,A G,A A,A G,A A,G A
259,1,A G,G G,A A,G A,A A,A G,G G,G A,G A,A A,G G,G G,G G,C A,G G
260,1,A A,G G,A A,G A,A A,G G,G G,A A,G A,A G,A G,G A,G G,C A,G G


In [10]:
test

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,1,A G,G G,A A,G A,A A,A G,G G,G A,G A,A G,A G,G A,G G,C A,G A
1,2,G G,A G,C C,G G,C C,A A,A A,A A,A A,G G,A G,A A,A A,A A,A A
2,2,G G,A G,A A,A A,C A,A G,A A,A A,A A,A G,A A,G A,G G,A A,G G
3,2,G G,A G,C A,A A,C C,A A,A A,A A,A A,G G,A A,G A,A G,A A,A A
4,1,A A,G G,A A,G G,A A,G G,G G,A A,G G,A G,G G,G A,G G,A A,G G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,2,A G,G G,C C,A A,C A,A G,A A,G G,A A,G G,G G,A A,A A,A A,G A
171,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,A G,A A,A A,A G,A A,G A
172,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,G G,A G,A A,A G,A A,G G
173,2,A G,G G,C A,G A,C C,G G,A A,G A,A A,G G,A G,A A,A A,A A,A A


In [25]:
train_x['trait-1'] = ''
train_x['trait-2'] = ''
test_x['trait-1'] = ''
test_x['trait-2'] = ''

In [26]:
for i in range(262):
  if train_x['trait'][i] == 1:
    train_x['trait-1'][i] = 1
    train_x['trait-2'][i] = 0
  if train_x['trait'][i] == 2:
    train_x['trait-1'][i] = 0
    train_x['trait-2'][i] = 1   

<ipython-input-26-26b8b1f878cc>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x['trait-1'][i] = 0
<ipython-input-26-26b8b1f878cc>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x['trait-2'][i] = 1
<ipython-input-26-26b8b1f878cc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x['trait-1'][i] = 1
<ipython-input-26-26b8b1f878cc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [27]:
train_x

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,trait-1,trait-2
0,2,G G,A G,A A,G A,C A,A A,A A,G G,A A,G G,A G,A A,A A,A A,A A,0,1
1,2,A G,A G,C A,A A,A A,A G,A A,G A,A A,A G,A A,G A,G G,A A,A A,0,1
2,2,G G,G G,A A,G A,C C,G G,A A,G A,G A,A G,A A,A A,A A,A A,A A,0,1
3,1,A A,G G,A A,G A,A A,G G,G G,A A,G G,A G,G G,G G,G G,A A,G G,1,0
4,2,G G,G G,C C,A A,C C,A A,A A,A A,A A,G G,A A,A A,A G,A A,G A,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,2,A G,A G,A A,G A,C C,A G,A A,G A,A A,G G,A G,G A,A A,A A,A A,0,1
258,2,G G,A A,C A,A A,A A,A G,G A,G A,A A,A G,A G,A A,A G,A A,G A,0,1
259,1,A G,G G,A A,G A,A A,A G,G G,G A,G A,A A,G G,G G,G G,C A,G G,1,0
260,1,A A,G G,A A,G A,A A,G G,G G,A A,G A,A G,A G,G A,G G,C A,G G,1,0


In [28]:
for i in range(175):
  if test_x['trait'][i] == 1:
    test_x['trait-1'][i] = 1
    test_x['trait-2'][i] = 0
  if test_x['trait'][i] == 2:
    test_x['trait-1'][i] = 0
    test_x['trait-2'][i] = 1   

<ipython-input-28-3eedb6abb8b3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x['trait-1'][i] = 1
<ipython-input-28-3eedb6abb8b3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x['trait-2'][i] = 0
<ipython-input-28-3eedb6abb8b3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x['trait-1'][i] = 0
<ipython-input-28-3eedb6abb8b3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [29]:
test_x

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,trait-1,trait-2
0,1,A G,G G,A A,G A,A A,A G,G G,G A,G A,A G,A G,G A,G G,C A,G A,1,0
1,2,G G,A G,C C,G G,C C,A A,A A,A A,A A,G G,A G,A A,A A,A A,A A,0,1
2,2,G G,A G,A A,A A,C A,A G,A A,A A,A A,A G,A A,G A,G G,A A,G G,0,1
3,2,G G,A G,C A,A A,C C,A A,A A,A A,A A,G G,A A,G A,A G,A A,A A,0,1
4,1,A A,G G,A A,G G,A A,G G,G G,A A,G G,A G,G G,G A,G G,A A,G G,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,2,A G,G G,C C,A A,C A,A G,A A,G G,A A,G G,G G,A A,A A,A A,G A,0,1
171,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,A G,A A,A A,A G,A A,G A,0,1
172,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,G G,A G,A A,A G,A A,G G,0,1
173,2,A G,G G,C A,G A,C C,G G,A A,G A,A A,G G,A G,A A,A A,A A,A A,0,1


In [30]:
class_le = preprocessing.LabelEncoder()
snp_le = preprocessing.LabelEncoder()
snp_col = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]

In [31]:
snp_data = []
for col in snp_col:
    snp_data += list(train_x[col].values)

In [32]:
train_y = class_le.fit_transform(train_y)
snp_le.fit(snp_data)

LabelEncoder()

In [33]:
for col in train_x.columns:
    if col in snp_col:
        train_x[col] = snp_le.transform(train_x[col])
        test_x[col] = snp_le.transform(test_x[col])

In [34]:
train_x

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,trait-1,trait-2
0,2,5,1,0,4,2,0,0,5,0,5,1,0,0,0,0,0,1
1,2,1,1,2,0,0,1,0,4,0,1,0,4,5,0,0,0,1
2,2,5,5,0,4,3,5,0,4,4,1,0,0,0,0,0,0,1
3,1,0,5,0,4,0,5,5,0,5,1,5,5,5,0,5,1,0
4,2,5,5,3,0,3,0,0,0,0,5,0,0,1,0,4,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,2,1,1,0,4,3,1,0,4,0,5,1,4,0,0,0,0,1
258,2,5,0,2,0,0,1,4,4,0,1,1,0,1,0,4,0,1
259,1,1,5,0,4,0,1,5,4,4,0,5,5,5,2,5,1,0
260,1,0,5,0,4,0,5,5,0,4,1,1,4,5,2,5,1,0


In [39]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   SNP_01   262 non-null    int64 
 1   SNP_02   262 non-null    int64 
 2   SNP_03   262 non-null    int64 
 3   SNP_04   262 non-null    int64 
 4   SNP_05   262 non-null    int64 
 5   SNP_06   262 non-null    int64 
 6   SNP_07   262 non-null    int64 
 7   SNP_08   262 non-null    int64 
 8   SNP_09   262 non-null    int64 
 9   SNP_10   262 non-null    int64 
 10  SNP_11   262 non-null    int64 
 11  SNP_12   262 non-null    int64 
 12  SNP_13   262 non-null    int64 
 13  SNP_14   262 non-null    int64 
 14  SNP_15   262 non-null    int64 
 15  trait-1  262 non-null    object
 16  trait-2  262 non-null    object
dtypes: int64(15), object(2)
memory usage: 34.9+ KB


In [40]:
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, LabelEncoder

In [41]:
le = LabelEncoder()

In [42]:
train_x['trait-1'] = le.fit_transform(train_x['trait-1'])
train_x['trait-2'] = le.fit_transform(train_x['trait-2'])
test_x['trait-1'] = le.fit_transform(test_x['trait-1'])
test_x['trait-2'] = le.fit_transform(test_x['trait-2'])

In [35]:
train_x.drop(labels='trait',axis=1,inplace=True)
test_x.drop(labels='trait',axis=1,inplace=True)

In [43]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier()

lgb.fit(train_x, train_y)

LGBMClassifier()

In [44]:
preds = lgb.predict(test_x)
print('Done.')

Done.


In [45]:
submit = pd.read_csv('/content/sample_submission.csv')

In [46]:
submit['class'] = class_le.inverse_transform(preds)

In [47]:
submit.to_csv('./submit.csv', index=False)